# Hyperparameter Tuning


In [1]:
import numpy as np

import os
from pathlib import Path

from nvita.utils import open_json
from nvita.models.data import SplittedTSData
from nvita.models.cnn import CNN
from nvita.models.lstm import LSTM
from nvita.models.gru import GRU
from nvita.models.rf import RF
import nvita.models.train as mt


d:\Programming\IDE\Anaconda\envs\hons\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH_ROOT = Path(os.getcwd()).parent.absolute()

my_metadata = open_json(os.path.join(
    PATH_ROOT, "experiments", "metadata.json"))
print(my_metadata)

{'seeds': ['2210', '9999', '58361', '789789', '1111111'], 'data': ['Electricity', 'NZTemp', 'CNYExch', 'Oil'], 'y_col_name': ['Consumption', 'Auckland', 'Close', 'Close'], 'window_size': ['4', '3', '7', '7'], 'models': ['CNN', 'LSTM', 'GRU', 'RF'], 'attacks': ['FGSM', 'BIM', 'nVITA', 'fullVITA']}


In [3]:
SEEDS = my_metadata["seeds"]
DATA_NAMES =  my_metadata["data"]
Y_COL_NAMES = my_metadata["y_col_name"]
WINDOW_SIZES = my_metadata["window_size"]
MODELS = my_metadata["models"]
TEST_SIZE = 100
VALID_PER = 0.2

## Split All Dataset for All Seeds and Save them on disk

In [4]:
for data_ind in range(len(DATA_NAMES)):
    for seed in SEEDS:
        path_df = os.path.join(PATH_ROOT, "data", "clean_data", DATA_NAMES[data_ind] +".csv")

        s_data = SplittedTSData(path_df, DATA_NAMES[data_ind], Y_COL_NAMES[data_ind], int(WINDOW_SIZES[data_ind]), int(seed))
        s_data.train_valid_test_split(TEST_SIZE, VALID_PER)
        s_data.save_splitted_data(PATH_ROOT)


File c:\Users\markc\Working_Space\nvita\results\splitted_data\df_Electricity_seed_2210.pkl has already existed!
File c:\Users\markc\Working_Space\nvita\results\splitted_data\df_Electricity_seed_9999.pkl has already existed!
File c:\Users\markc\Working_Space\nvita\results\splitted_data\df_Electricity_seed_58361.pkl has already existed!
File c:\Users\markc\Working_Space\nvita\results\splitted_data\df_Electricity_seed_789789.pkl has already existed!
File c:\Users\markc\Working_Space\nvita\results\splitted_data\df_Electricity_seed_1111111.pkl has already existed!
File c:\Users\markc\Working_Space\nvita\results\splitted_data\df_NZTemp_seed_2210.pkl has already existed!
File c:\Users\markc\Working_Space\nvita\results\splitted_data\df_NZTemp_seed_9999.pkl has already existed!
File c:\Users\markc\Working_Space\nvita\results\splitted_data\df_NZTemp_seed_58361.pkl has already existed!
File c:\Users\markc\Working_Space\nvita\results\splitted_data\df_NZTemp_seed_789789.pkl has already existed!
Fil

In [5]:
def grid_search_cnn(s_d):
    max_epochs = [100, 250, 500]
    learning_rate = [0.001, 0.005, 0.01]
    window_size = s_data.window_size
    #module__window_size = window_size
    module__conv_out_and_f1 = [64, 128, 256]
    f0 = s_d.X_train.shape[2]
    #module__f1 = [64, 128, 256, 512]
    module__f2 = [32, 64 ,128]
    out = s_d.y_train.shape[1]

    best_score = np.Inf
    best_cnn_paras = []
    for epoch in max_epochs:
        for lr in learning_rate: 
            for c in module__conv_out_and_f1:
                f1 = c
                for f2 in module__f2:
                    model = CNN(window_size, c, f0, f1, f2, out)
                    mt.train(model, lr, epoch, s_d.X_train, s_d.y_train)
                    score = np.mean(mt.evaluate(model, s_d.X_valid, s_d.y_valid)**2)
                    if score < best_score:
                        # Smaller score MSE indicates better performance
                        best_score = score
                        best_cnn_paras = [epoch, lr, window_size, c, f0, f1, f2, out]
    print("Best Score for " + model_name + " is " + str(best_score) + " on dataset " + str(s_d.df_name))
    return best_cnn_paras

def grid_search_rnn(s_d, model_name = "LSTM"):
    max_epochs = [100, 250, 500]
    learning_rate = [0.001, 0.005, 0.01]
    #module__window_size = window_size
    input_dim = s_d.X_train.shape[2]
    module__hidden_dim = [64, 128, 256]
    module__num_layers = [1, 2, 4]
    output_dim = s_d.y_train.shape[1]

    best_score = np.Inf
    best_rnn_paras = []
    for epoch in max_epochs:
        for lr in learning_rate: 
            for hidden_dim in module__hidden_dim:
                for num_layers in module__num_layers:
                    if model_name == "LSTM":
                        model = LSTM(input_dim, hidden_dim, num_layers, output_dim)
                    elif model_name == "GRU":
                        model = GRU(input_dim, hidden_dim, num_layers, output_dim)
                    mt.train(model, lr, epoch, s_d.X_train, s_d.y_train)
                    score = np.mean(mt.evaluate(model, s_d.X_valid, s_d.y_valid)**2)
                    if score < best_score:
                        # Smaller score MSE indicates better performance
                        best_score = score
                        best_rnn_paras = [epoch, lr, input_dim, hidden_dim, num_layers, output_dim]
    print("Best Score for " + model_name + " is " + str(best_score) + " on dataset " + str(s_d.df_name))
    return best_rnn_paras

def grid_search_rf(s_d):
    module__n_estimators = [100, 250, 500, 1000]

    best_score = np.Inf
    best_rf_paras = []
    for n_estimators in module__n_estimators:
        model = RF(n_estimators)
        model.fit(s_d.X_train, s_d.y_train)
        score = np.mean(mt.evaluate(model, s_d.X_valid, s_d.y_valid)**2)
        if score < best_score:
            # Smaller score MSE indicates better performance
            best_score = score
            best_rf_paras = [n_estimators]
    print("Best Score for " + model_name + " is " + str(best_score) + " on dataset " + str(s_d.df_name))
    return best_rf_paras

In [6]:
best_cnn_paras_dict = dict()
bset_lstm_paras_dict = dict()
best_gru_paras_dict = dict()
best_rf_paras_dict = dict()
for data_ind in range(len(DATA_NAMES)):
    seed = SEEDS[0]
    s_data = SplittedTSData()
    s_data = s_data.load_splitted_data(PATH_ROOT, DATA_NAMES[data_ind], seed)
    for model_name in MODELS:
        if model_name == "CNN":
            best_cnn_paras_dict[DATA_NAMES[data_ind]] = grid_search_cnn(s_data)
        elif model_name == "LSTM":
            bset_lstm_paras_dict[DATA_NAMES[data_ind]] = grid_search_rnn(s_data, model_name)
        elif model_name == "GRU":
            best_gru_paras_dict[DATA_NAMES[data_ind]] = grid_search_rnn(s_data, model_name)
        elif model_name == "RF":
            best_rf_paras_dict[DATA_NAMES[data_ind]] = grid_search_rf(s_data)

Best Score for CNN is 0.008788654 on dataset Electricity
Best Score for LSTM is 0.0103443805 on dataset Electricity
Best Score for GRU is 0.010461737 on dataset Electricity
Best Score for RF is 0.008242441980566248 on dataset Electricity
Best Score for CNN is 0.00413001 on dataset NZTemp
